# Customer Segmentation using RFM Analysis

## Business Problem Statement

The company treats all customers equally despite significant differences
in purchasing behavior and profitability. Leadership needs a data-driven
way to identify high-value, at-risk, and low-engagement customers.

## Why This Matters

Customer segmentation enables:
- Targeted marketing campaigns
- Smarter discount allocation
- Better customer retention
- Higher lifetime value

In [1]:
import pandas as pd

df = pd.read_csv("../data/processed/featured_data.csv")
df['Order Date'] = pd.to_datetime(df['Order Date'])

In [2]:
snapshot_date = df['Order Date'].max() + pd.Timedelta(days=1)

rfm = df.groupby('Customer ID').agg({
    'Order Date': lambda x: (snapshot_date - x.max()).days,
    'Order ID': 'nunique',
    'Sales': 'sum'
}).reset_index()

rfm.columns = ['Customer ID', 'Recency', 'Frequency', 'Monetary']
rfm.head()

,Customer ID,Recency,Frequency,Monetary
0,AA-10315,185,5,5563.560
1,AA-10375,20,9,1056.390
2,AA-10480,260,4,1790.512
3,AA-10645,56,6,5086.935
4,AB-10015,416,3,886.156


In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['Recency', 'Frequency', 'Monetary']])

In [4]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=4, random_state=42)
rfm['Segment'] = kmeans.fit_predict(rfm_scaled)

In [5]:
segment_summary = rfm.groupby('Segment').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': 'mean',
    'Customer ID': 'count'
}).rename(columns={'Customer ID': 'Num_Customers'})

segment_summary

,Recency,Frequency,Monetary,Num_Customers
Segment,,,,
0,72.741611,8.516779,3322.222985,298
1,101.197015,4.731343,1669.688290,335
2,123.718750,8.296875,9479.545687,64
3,559.489583,3.697917,1470.228226,96


In [6]:
import joblib

rfm.to_csv("../data/processed/customer_segments.csv", index=False)
joblib.dump(kmeans, "../models/customer_segmentation.pkl")

['../models/customer_segmentation.pkl']

In [7]:
rfm.to_csv("../data/processed/customer_rfm.csv", index=False)

In [8]:
pd.read_csv("../data/processed/customer_rfm.csv").columns
df.head(5)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Sales,Quantity,Discount,Profit,order_year,order_month,order_quarter,order_dayofweek,profit_margin,is_discounted
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,261.9600,2,0.00,41.9136,2016,11,4,1,0.1600,0
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,731.9400,3,0.00,219.5820,2016,11,4,1,0.3000,0
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,14.6200,2,0.00,6.8714,2016,6,2,6,0.4700,0
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,957.5775,5,0.45,-383.0310,2015,10,4,6,-0.4000,1
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,22.3680,2,0.20,2.5164,2015,10,4,6,0.1125,1


In [9]:
rfm_summary = (
    rfm.groupby("Segment")
    .agg(
        Customers=("Customer ID", "count"),
        Avg_Recency=("Recency", "mean"),
        Avg_Frequency=("Frequency", "mean"),
        Avg_Monetary=("Monetary", "mean"),
        Total_Revenue=("Monetary", "sum")
    )
    .reset_index()
)

rfm_summary.to_csv(
    "../data/processed/customer_segment_summary.csv",
    index=False
)


In [10]:

rfm["CLV"] = rfm["Monetary"] * rfm["Frequency"]

# Churn proxy (low recency + low frequency)
rfm["Churn_Risk"] = (
    (rfm["Recency"] > rfm["Recency"].quantile(0.75)) &
    (rfm["Frequency"] < rfm["Frequency"].quantile(0.25))
).astype(int)

# Revenue concentration
top_20_pct = rfm.sort_values("Monetary", ascending=False).head(int(0.2 * len(rfm)))
revenue_concentration = top_20_pct.Monetary.sum() / rfm.Monetary.sum()

# Segment actions
action_map = {
    0: "Retain & Upsell",
    1: "Loyal – Grow Share",
    2: "Re-engage Immediately",
    3: "New – Nurture"
}

rfm["Business_Action"] = rfm["Segment"].map(action_map)

rfm.to_csv("data/processed/customer_business_insights.csv", index=False)

pd.DataFrame([{
    "total_customers": rfm["Customer ID"].nunique(),
    "revenue_concentration": revenue_concentration,
    "high_risk_customers": rfm["Churn_Risk"].sum()
}]).to_csv("data/processed/customer_kpis.csv", index=False)


## Executive Summary

Customer segmentation reveals distinct groups with different purchasing
behaviors. High-frequency, high-monetary customers drive a disproportionate
share of revenue and should be prioritized for retention and loyalty programs.